In [44]:
import pandas as pd
from google.cloud import bigquery

In [45]:
new_output_df = pd.read_csv('research_papers_new_processing.csv', index_col=0)
bq_client = bigquery.Client(project='evident-data-dev')
old_output_df = bq_client.list_rows('evident-data-dev.curated_google_scholar.banking_research').to_dataframe()

qa_df = pd.read_csv('research_papers_qa_dataset.csv', index_col=0)


c:\Users\lucas\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
c:\Users\lucas\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
C:\Users\lucas\AppData\Local\Temp\ipykernel_27520\3503487757.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  qa_df = pd.read_csv('research_papers_qa_dataset.csv', index_col=0)


In [46]:
mask = ~old_output_df['paper_url'].isin(new_output_df['paper_url'])

old_not_new_df = old_output_df[mask]
old_not_new_df.head()

,run_month,company_id,company_name,author_url,author_name,paper_url,paper_title,paper_date,paper_citations
10,2025-02-28,1002,Commerzbank,/citations?hl=en&user=TDAL0_EAAAAJ,Ludvík Tesař,/citations?view_op=view_citation&hl=en&user=TD...,gaussian mixture model for texture-based medic...,2015-01-01,0
25,2025-02-28,1003,Santander,/citations?hl=en&user=mZn3e8wAAAAJ,Jose M. Leiva Murillo,/citations?view_op=view_citation&hl=en&user=mZ...,algorithms for maximum-likelihood bandwidth,2015-01-01,0
39,2025-02-28,1004,JPMorganChase,/citations?hl=en&user=-NOTfHIAAAAJ,Sairamesh Konidala,/citations?view_op=view_citation&hl=en&user=-N...,training ai models on sensitive data-the feder...,2015-01-01,17
40,2025-02-28,1004,JPMorganChase,/citations?hl=en&user=-NOTfHIAAAAJ,Sairamesh Konidala,/citations?view_op=view_citation&hl=en&user=-N...,designing and implementing efficient data pipe...,2015-01-01,4
71,2025-02-28,1004,JPMorganChase,/citations?hl=en&user=1WkDG2UAAAAJ,Colin Targonski,/citations?view_op=view_citation&hl=en&user=1W...,a generalized deep learning approach for local...,2019-01-01,87


In [53]:
companies_to_check = ["UBS", "ING"]

In [54]:
lhs_cols = ["company_name", "paper_url"]
rhs_cols = ["authors", 'url', "title", "publication_date", "is_patent", "tags",
            "excluded_tags", "keyword_tags", "has_publication_info"]

In [56]:
ubs_with_qa = pd.merge(
    old_not_new_df[old_not_new_df['company_name'] == companies_to_check[0]][lhs_cols],
    qa_df[rhs_cols],
    left_on='paper_url',
    right_on = 'url',
    how='left').drop(columns=['url'])

ubs_with_qa

,company_name,paper_url,authors,title,publication_date,is_patent,tags,excluded_tags,keyword_tags,has_publication_info
0,UBS,/citations?view_op=view_citation&hl=en&user=29...,"Mariusz Giergiel Aleksander Budziński, Tomasz ...",concept of swarm specialized distributed local...,2016/1,False,[],[],[],True
1,UBS,/citations?view_op=view_citation&hl=en&user=6T...,"MR Stefan, Arnfried Ossen",clustering in weight space of feedforward nets,NaN,False,['ai_keywords'],[],['clustering'],False
2,UBS,/citations?view_op=view_citation&hl=en&user=6T...,Arnfried Ossen,department of computer science technical unive...,NaN,False,['ai_keywords'],[],"['supervised learning', 'unsupervised learning...",False
3,UBS,/citations?view_op=view_citation&hl=en&user=6T...,"MR Stefan, Arnfried Ossen",performance evaluation of feedforward networks...,NaN,False,['ai_keywords'],[],"['neural network', 'clustering', 'neural net']",False
4,UBS,/citations?view_op=view_citation&hl=en&user=kl...,Anson Ho,"future-proof: monitoring the development, depl...",NaN,False,['ai_keywords'],[],"['artificial intelligence', ' ai ']",False
5,UBS,/citations?view_op=view_citation&hl=en&user=kl...,Anson Ho,"future-proof: monitoring the development, depl...",NaN,False,['ai_keywords'],[],"['artificial intelligence', ' ai ']",False
6,UBS,/citations?view_op=view_citation&hl=en&user=rC...,Georgios Touloupas,deep learning for analyzing atmospheric clouds,2018,False,['ai_keywords'],['excluded_keywords'],"['machine learning', 'deep learning', 'automate']",True
